In [1]:
from sklearn.preprocessing import FunctionTransformer

from reskit.norms import binar_norm, wbysqdist
from reskit.norms import spectral_norm

from reskit.features import degrees,  pagerank

from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier 
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from reskit.core import Transformer, Pipeliner

def orig(x):
    return x

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Функция считывания данных

In [2]:
def get_autism(path_to_read='Data/dti/', distances=True):
    def get_autism_distances(loc_name):
        with open(loc_name, 'r') as f:
            read_data = f.readlines()

        read_data = pd.DataFrame(
            np.array([np.array(item[:-1].split()).astype(int) for item in read_data]))

        return read_data

    def get_distance_matrix(coords):
        if type(coords) == pd.core.frame.DataFrame:
            coords = coords.values
        elif type(coords) != np.ndarray:
            print('Provide either pandas df or numpy array!')
            return -1

        shape = len(coords)
        dist_matrix = np.zeros((shape, shape))
        del shape
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                dist_matrix[i, j] = np.linalg.norm(coords[i, :] - coords[j, :])
                dist_matrix[j, i] = dist_matrix[i, j]
        return dist_matrix

    target_vector = []  # this will be a target vector (diagnosis)
    matrices = []  # this will be a list of connectomes
    all_files = sorted(os.listdir(path_to_read))
    matrix_files = [
        item for item in all_files if 'DTI_connectivity' in item and 'All' not in item]
    distance_files = [
        item for item in all_files if 'DTI_region_xyz_centers' in item and 'All' not in item]

    # for each file in a sorted (!) list of files:
    for filename in matrix_files:

        A_dataframe = pd.read_csv(
            path_to_read + filename, sep='   ', header=None, engine='python')
        A = A_dataframe.values  # we will use a list of numpy arrays, NOT pandas dataframes
        matrices.append(A)# append a matrix to our list
        if "ASD" in filename:
            target_vector.append(1)
        elif "TD" in filename:
            target_vector.append(0)
    asd_dict = {}
    asd_dict['X'] = np.array(matrices)
    asd_dict['y'] = np.array(target_vector)
    if distances:
        dist_matrix_list = []
        for item in distance_files:
            # print(item)
            cur_coord = get_autism_distances(path_to_read + item)
            cur_dist_mtx = get_distance_matrix(cur_coord)
            dist_matrix_list += [cur_dist_mtx]

        asd_dict['dist'] = np.array(dist_matrix_list)

    return asd_dict


### Функция понижения ранга матрицы

In [14]:
def matrix_eig(data, k = 0):
    new_data = {}
    new_data['y'] = data['y']
    new_data['dist'] = data['dist']
    new_data['X'] = np.zeros(shape = (data['X'].shape[0], data['X'].shape[1], data['X'].shape[1] - k))
    for i in np.arange(data['X'].shape[0]):
        curs, vecs = np.linalg.eig(data['X'][i])
        indeces_del = range(curs.size)[(curs.size - k):]
        new_data['X'][i] = np.delete(vecs*curs, indeces_del, axis=1).astype('float')
    return new_data

## Сделаем один пайплайн

In [15]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

In [23]:
data = 'Data/dti/'
data = Transformer(get_autism).fit_transform(data)

In [24]:
data = Transformer(matrix_eig, {'data': data, 'k': 40}).fit_transform(data)
data = Transformer(degrees, collect=['degrees']).fit_transform(data)

In [25]:
print data

(array([[  6.09542131e+03,  -3.70137733e+03,  -7.48535401e+02, ...,
         -2.66765237e+00,  -2.95576538e+00,  -4.07359649e+00],
       [ -6.25772292e+03,  -4.79222785e+02,   3.08833720e+02, ...,
         -1.16238663e+00,  -3.30382395e+00,   2.26545427e+01],
       [  4.27905871e+03,   4.43526218e+03,  -2.86224330e+03, ...,
          1.14853184e+00,  -6.99566351e-01,  -1.06067595e-02],
       ..., 
       [  5.05319819e+03,   2.91379096e+03,   4.05156059e+03, ...,
         -9.24105067e+00,   1.59543422e+01,  -1.17473146e+01],
       [  6.86404286e+03,   2.14342540e+03,   1.43555795e+03, ...,
          7.02563437e+00,  -7.16864877e+00,  -4.41833101e+00],
       [  5.08492048e+03,   3.47835743e+03,  -1.42710301e+03, ...,
          4.91268675e+00,   2.62841057e+00,  -3.20953611e+00]]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [26]:
X, y = data 
print X.shape, y.shape

(94, 224) (94,)


In [27]:
steps = [('selector', VarianceThreshold()), ('scaler', MinMaxScaler()), ('classifier', LogisticRegression())] 
pipeline = Pipeline(steps)

In [28]:
param_grid = dict(classifier__penalty=['l1', 'l2'])
scoring = 'roc_auc'
grid_clf = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=scoring, n_jobs=-1, cv=grid_cv)
grid_clf.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=0, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('selector', VarianceThreshold(threshold=0.0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'classifier__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [29]:
steps[-1] = steps[-1][0], grid_clf.best_estimator_
pipeline = Pipeline(steps)
scores = cross_val_score(pipeline, X, y, scoring=scoring, cv=eval_cv, n_jobs=-1)
np.mean(scores), np.std(scores)

(0.55266666666666653, 0.1564736399525492)

### Приведу некоторые результаты для различных К

|       scores       |         std        |  k |
|      :------:      |        :---:       | :-:|
| 0.59233333333333338| 0.12568081264324588|  0 |
| 0.59233333333333338| 0.12568081264324588|  1 | 
| 0.59233333333333338| 0.12568081264324588|  2 | 
| 0.59433333333333338| 0.13016271867679058| 10 |
| 0.61133333333333328| 0.16110865898517063| 20 |
| 0.63233333333333341| 0.16096272860510288| 30 |
| 0.63233333333333341| 0.16096272860510288| 40 |
| 0.63233333333333341| 0.16096272860510288| 45 |
| 0.54400000000000004| 0.27115309328864384| 50 |
| 0.21695468036742913| 0.22945079356294823| 75 |
| 0.49466666666666664| 0.21695468036742913| 100|

## Попробуем сделать это, используя класс Papiliner

Здесь возникают проблемы, из-за того, что работая с один пайплайном мы явно можем задать парамеры функции. Тут же сделать это сложнее.

Есть несколько решений:  
1. Задавать k по дефолту в функции  
2. Задавать параметр data функции matrix_eig через стороннюю переменную, расчитанную ранее

На мой взгляд, лучши решением будет первый вариант. Поэтому на нем я и остановился

In [3]:
def orig_vec(data):
    matrices = []
    for i in  data['X']:
        matrices.append(np.hstack(i))
    data['X_vec'] = matrices
    return data

In [8]:
def matrix_eig_k(data, k = 30):
    new_data = {}
    new_data['y'] = data['y']
    new_data['dist'] = data['dist']
    new_data['X'] = np.zeros(shape = (data['X'].shape[0], data['X'].shape[1], data['X'].shape[1] - k))
    for i in np.arange(data['X'].shape[0]):
        curs, vecs = np.linalg.eig(data['X'][i])
        indeces_del = range(curs.size)[(curs.size - k):]
        new_data['X'][i] = np.delete(vecs, indeces_del, axis=1).astype('float')
    return orig_vec(new_data)

In [9]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

data = [('UCLAsource', Transformer(get_autism)),
        #('UCLAbaseline', Transformer(get_baseline))
       ]

#Only low_rank
weighters = [('origW', Transformer(orig)),
             #('binar', Transformer(binar_norm)),
             #('wbysqdist', Transformer(wbysqdist)),
             #('low_rank', Transformer(matrix_eig_k))
            ]


normalizers = [('origN', Transformer(orig)),
               #('spectral', Transformer(spectral_norm))
              ]

featurizers = [#('origF', Transformer(orig_vec, collect=['X_vec'])),
               #('degrees', Transformer(degrees, collect=['degrees']))
               ('low_rank', Transformer(matrix_eig_k, collect=['X_vec']))]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler()),
           ('origS', FunctionTransformer(orig))]

#For tests, don`t use XGB, it needs a lot of time
classifiers = [('LR', LogisticRegression()),
               ('RF', RandomForestClassifier()),
               ('SVC', SVC()),
               #('XGB', XGBClassifier(nthread=1)),
               ('SGD', SGDClassifier())]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

banned_combos = [#('UCLAsource', 'origN'),
                 #('UCLAsource', 'origF'),
                 ('UCLAbaseline', 'degrees'),
                 ('UCLAbaseline', 'binar'),
                 ('UCLAbaseline', 'wbysqdist'),
                 ('UCLAbaseline', 'spectral'),
                 ('UCLAbaseline', 'low_rank'),
                 ('LR', 'origS'),
                 ('SVC', 'origS'),
                 ('SGD', 'origS'),
                 ('RF', 'minmax'),
                 ('XGB', 'minmax')]

param_grid = dict(
    LR=dict(
        C=[0.01, 0.05, 0.1] + [0.05*i for i in range(3, 21)],
        max_iter=[50, 100, 500],
        penalty=['l1', 'l2']
    ),
    SGD=dict(
        alpha=[0.001, 0.01, 0.1, 0.5, 1.0],
        l1_ratio=[0, 0.2, 0.4, 0.6, 0.8, 1],
        loss=['hinge', 'log', 'modified_huber'],
        n_iter=[50, 100, 200],
        penalty=['elasticnet']
    ),
    SVC=dict(
        C=[0.0005, 0.001, 0.005, 0.01] + [i*0.05 for i in range(1,11)],
        degree=[2, 3, 4],
        kernel=['linear', 'poly', 'rbf', 'sigmoid'],
        max_iter=[50, 100, 150],
    ),
    RF=dict(
        criterion=['entropy', 'gini'],
        max_depth=[3, 5, 7, 10, 20],
        max_features=['log2', 'sqrt'] + [0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5, 1.0],
        n_estimators=[10, 50, 100, 200, 500]
    ),
    XGB=dict(
        colsample_bytree=[0.01] + [0.05*i for i in range(1,21)],
        learning_rate=[0.01*i for i in range(1,6)] + [0.05*i for i in range(2,11)],
        max_depth=[i for i in range(1,12)],
        n_estimators=[10, 50, 100, 200, 500],
        nthread=[1],
        reg_alpha=[0, 1],
        reg_lambda=[0, 1],
        subsample=[0.5, 0.7, 1]
    )
)

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,origW,origN,low_rank,var_threshold,minmax,LR
1,UCLAsource,origW,origN,low_rank,var_threshold,minmax,SVC
2,UCLAsource,origW,origN,low_rank,var_threshold,minmax,SGD
3,UCLAsource,origW,origN,low_rank,var_threshold,origS,RF


In [24]:
pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'], scoring=['roc_auc'])

Removed previous results file -- results.csv.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4


,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,UCLAsource,origW,origN,low_rank,var_threshold,minmax,LR,0.625,0.149066,"{'penalty': 'l1', 'C': 0.5, 'max_iter': 50}",0.608333,0.196736,[ 0.53333333 0.72 0.68 0.4 ...
1,UCLAsource,origW,origN,low_rank,var_threshold,minmax,SVC,0.412589,0.194786,"{'kernel': 'poly', 'C': 0.0005, 'max_iter': 50...",0.399,0.109768,[ 0.3 0.36 0.48 0.4 0.45 0.35 0.3 0....
2,UCLAsource,origW,origN,low_rank,var_threshold,minmax,SGD,0.703191,0.175191,"{'penalty': 'elasticnet', 'loss': 'log', 'alph...",0.611,0.17524,[ 0.5 0.48 0.68 0.5 1. 0.45 0.85 0....
3,UCLAsource,origW,origN,low_rank,var_threshold,origS,RF,0.698138,0.224581,"{'n_estimators': 200, 'max_features': 0.25, 'c...",0.599333,0.185074,[ 0.73333333 0.64 0.72 0.325 ...


In [10]:
def matrix_eig_10(data, k = 10):
    new_data = {}
    new_data['y'] = data['y']
    new_data['dist'] = data['dist']
    new_data['X'] = np.zeros(shape = (data['X'].shape[0], data['X'].shape[1], data['X'].shape[1] - k))
    for i in np.arange(data['X'].shape[0]):
        curs, vecs = np.linalg.eig(data['X'][i])
        indeces_del = range(curs.size)[(curs.size - k):]
        new_data['X'][i] = np.delete(vecs, indeces_del, axis=1).astype('float')
    return orig_vec(new_data)

def matrix_eig_30(data, k = 30):
    new_data = {}
    new_data['y'] = data['y']
    new_data['dist'] = data['dist']
    new_data['X'] = np.zeros(shape = (data['X'].shape[0], data['X'].shape[1], data['X'].shape[1] - k))
    for i in np.arange(data['X'].shape[0]):
        curs, vecs = np.linalg.eig(data['X'][i])
        indeces_del = range(curs.size)[(curs.size - k):]
        new_data['X'][i] = np.delete(vecs, indeces_del, axis=1).astype('float')
    return orig_vec(new_data)

def matrix_eig_50(data, k = 50):
    new_data = {}
    new_data['y'] = data['y']
    new_data['dist'] = data['dist']
    new_data['X'] = np.zeros(shape = (data['X'].shape[0], data['X'].shape[1], data['X'].shape[1] - k))
    for i in np.arange(data['X'].shape[0]):
        curs, vecs = np.linalg.eig(data['X'][i])
        indeces_del = range(curs.size)[(curs.size - k):]
        new_data['X'][i] = np.delete(vecs, indeces_del, axis=1).astype('float')
    return orig_vec(new_data)

In [11]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

data = [('UCLAsource', Transformer(get_autism)),
        #('UCLAbaseline', Transformer(get_baseline))
       ]

#Only low_rank
weighters = [('origW', Transformer(orig)),
             ('binar', Transformer(binar_norm)),
             ('wbysqdist', Transformer(wbysqdist)),
            ]


normalizers = [('origN', Transformer(orig)),
               ('spectral', Transformer(spectral_norm))
              ]

featurizers = [('origF', Transformer(orig_vec, collect=['X_vec'])),
               #('degrees', Transformer(degrees, collect=['degrees']))
               ('low_rank_10', Transformer(matrix_eig_10, collect=['X_vec'])),
               ('low_rank_30', Transformer(matrix_eig_30, collect=['X_vec'])),
               ('low_rank_50', Transformer(matrix_eig_50, collect=['X_vec']))]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler()),
           ('origS', FunctionTransformer(orig))]

#For tests, don`t use XGB, it needs a lot of time
classifiers = [('LR', LogisticRegression()),
               #('RF', RandomForestClassifier()),
               #('SVC', SVC()),
               #('XGB', XGBClassifier(nthread=1)),
               #('SGD', SGDClassifier())
              ]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

banned_combos = ['''
                 ('UCLAsource', 'origN'),
                 ('UCLAsource', 'origF'),
                 ('UCLAbaseline', 'degrees'),
                 ('UCLAbaseline', 'binar'),
                 ('UCLAbaseline', 'wbysqdist'),
                 ('UCLAbaseline', 'spectral'),
                 ('UCLAbaseline', 'low_rank'),
                 ('LR', 'origS'),
                 ('SVC', 'origS'),
                 ('SGD', 'origS'),
                 ('RF', 'minmax'),
                 ('XGB', 'minmax')
                 '''
                 
                 ]

param_grid = dict(
    LR=dict(
        C=[0.01, 0.05, 0.1] + [0.05*i for i in range(3, 21)],
        max_iter=[50, 100, 500],
        penalty=['l1', 'l2']
    ),
    SGD=dict(
        alpha=[0.001, 0.01, 0.1, 0.5, 1.0],
        l1_ratio=[0, 0.2, 0.4, 0.6, 0.8, 1],
        loss=['hinge', 'log', 'modified_huber'],
        n_iter=[50, 100, 200],
        penalty=['elasticnet']
    ),
    SVC=dict(
        C=[0.0005, 0.001, 0.005, 0.01] + [i*0.05 for i in range(1,11)],
        degree=[2, 3, 4],
        kernel=['linear', 'poly', 'rbf', 'sigmoid'],
        max_iter=[50, 100, 150],
    ),
    RF=dict(
        criterion=['entropy', 'gini'],
        max_depth=[3, 5, 7, 10, 20],
        max_features=['log2', 'sqrt'] + [0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5, 1.0],
        n_estimators=[10, 50, 100, 200, 500]
    ),
    XGB=dict(
        colsample_bytree=[0.01] + [0.05*i for i in range(1,21)],
        learning_rate=[0.01*i for i in range(1,6)] + [0.05*i for i in range(2,11)],
        max_depth=[i for i in range(1,12)],
        n_estimators=[10, 50, 100, 200, 500],
        nthread=[1],
        reg_alpha=[0, 1],
        reg_lambda=[0, 1],
        subsample=[0.5, 0.7, 1]
    )
)

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,origW,origN,origF,var_threshold,minmax,LR
1,UCLAsource,origW,origN,origF,var_threshold,origS,LR
2,UCLAsource,origW,origN,low_rank_10,var_threshold,minmax,LR
3,UCLAsource,origW,origN,low_rank_10,var_threshold,origS,LR
4,UCLAsource,origW,origN,low_rank_30,var_threshold,minmax,LR
5,UCLAsource,origW,origN,low_rank_30,var_threshold,origS,LR
6,UCLAsource,origW,origN,low_rank_50,var_threshold,minmax,LR
7,UCLAsource,origW,origN,low_rank_50,var_threshold,origS,LR
8,UCLAsource,origW,spectral,origF,var_threshold,minmax,LR
9,UCLAsource,origW,spectral,origF,var_threshold,origS,LR


In [12]:
pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'], scoring=['roc_auc'])

Removed previous results file -- results.csv.
Line: 1/48
Line: 2/48
Line: 3/48
Line: 4/48
Line: 5/48
Line: 6/48
Line: 7/48
Line: 8/48
Line: 9/48
Line: 10/48
Line: 11/48
Line: 12/48
Line: 13/48
Line: 14/48
Line: 15/48
Line: 16/48
Line: 17/48
Line: 18/48
Line: 19/48
Line: 20/48
Line: 21/48
Line: 22/48
Line: 23/48
Line: 24/48
Line: 25/48
Line: 26/48
Line: 27/48
Line: 28/48
Line: 29/48
Line: 30/48
Line: 31/48
Line: 32/48
Line: 33/48


/home/nikita/.local/lib/python2.7/site-packages/reskit/norms.py:256: RuntimeWarning: invalid value encountered in divide
  weighted_X = X / dist ** 2


Line: 34/48
Line: 35/48
Line: 36/48
Line: 37/48
Line: 38/48
Line: 39/48
Line: 40/48
Line: 41/48
Line: 42/48
Line: 43/48
Line: 44/48
Line: 45/48
Line: 46/48
Line: 47/48
Line: 48/48


,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,UCLAsource,origW,origN,origF,var_threshold,minmax,LR,0.607801,0.157715,"{'penalty': 'l2', 'C': 0.1, 'max_iter': 50}",0.593333,0.159332,[ 0.63333333 0.56 0.64 0.85 ...
1,UCLAsource,origW,origN,origF,var_threshold,origS,LR,0.596454,0.195455,"{'penalty': 'l1', 'C': 0.05, 'max_iter': 50}",0.580667,0.158527,[ 0.66666667 0.6 0.44 0.75 ...
2,UCLAsource,origW,origN,low_rank_10,var_threshold,minmax,LR,0.61844,0.175673,"{'penalty': 'l1', 'C': 0.4, 'max_iter': 100}",0.638333,0.121676,[ 0.53333333 0.68 0.72 0.5 ...
3,UCLAsource,origW,origN,low_rank_10,var_threshold,origS,LR,0.608333,0.133602,"{'penalty': 'l1', 'C': 0.75, 'max_iter': 50}",0.543333,0.240278,[ 0.53333333 0.4 0.4 0.35 ...
4,UCLAsource,origW,origN,low_rank_30,var_threshold,minmax,LR,0.625532,0.137521,"{'penalty': 'l1', 'C': 0.5, 'max_iter': 500}",0.624333,0.194634,[ 0.53333333 0.68 0.68 0.4 ...
5,UCLAsource,origW,origN,low_rank_30,var_threshold,origS,LR,0.608333,0.133602,"{'penalty': 'l1', 'C': 0.75, 'max_iter': 50}",0.543333,0.240278,[ 0.53333333 0.4 0.4 0.35 ...
6,UCLAsource,origW,origN,low_rank_50,var_threshold,minmax,LR,0.616312,0.181489,"{'penalty': 'l1', 'C': 0.4, 'max_iter': 50}",0.656667,0.128374,[ 0.56666667 0.68 0.72 0.5 ...
7,UCLAsource,origW,origN,low_rank_50,var_threshold,origS,LR,0.608333,0.133602,"{'penalty': 'l1', 'C': 0.75, 'max_iter': 50}",0.543333,0.240278,[ 0.53333333 0.4 0.4 0.35 ...
8,UCLAsource,origW,spectral,origF,var_threshold,minmax,LR,0.579433,0.187257,"{'penalty': 'l2', 'C': 0.1, 'max_iter': 50}",0.557667,0.175297,[ 0.56666667 0.56 0.6 0.75 ...
9,UCLAsource,origW,spectral,origF,var_threshold,origS,LR,0.574468,0.17153,"{'penalty': 'l2', 'C': 0.5, 'max_iter': 50}",0.586,0.144928,[ 0.5 0.52 0.44 0.85 0.8 0.65 0.35 0....
